## Import thư viện PyTorch

In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchviz import make_dot

## Thiết lập device

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Đặt các siêu tham số (Hyper-parameters)

In [3]:
# Số lượng epochs
num_epochs = 5

# Số lượng lớp
num_classes = 10

# Kích thước mỗi batch
batch_size = 100

# Tốc độ học - Learning rate
learning_rate = 0.001

## Load bộ dữ liệu MNTST

In [4]:
# Dữ liệu tập huấn luyện
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

# Dữ liệu tập kiểm tra
test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

## DataLoader

In [5]:
# DataLoader cho tập huấn luyện
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

# DataLoader cho tập kiểm tra
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

## Mô hình Convolutional neural network (two convolutional layers)

In [6]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [7]:
model = ConvNet(num_classes).to(device)
model

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=1568, out_features=10, bias=True)
)

## Hàm mất mát (loss function) và bộ tối ưu hóa (optimizer)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Huấn luyện mô hình

In [9]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Chuyển dữ liệu sang device
        images = images.to(device)
        labels = labels.to(device)
        
        # Lan truyền tiến (Forward pass) và tính toán mất mát
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Lan truyền ngược (Backward pass) và tối ưu
        ## Đặt zero_grad để không tính toán đạo hàm
        optimizer.zero_grad()
        ## Gọi backward()
        loss.backward()
        ## Tối ưu
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.0799
Epoch [1/5], Step [200/600], Loss: 0.1546
Epoch [1/5], Step [300/600], Loss: 0.0685
Epoch [1/5], Step [400/600], Loss: 0.0266
Epoch [1/5], Step [500/600], Loss: 0.0746
Epoch [1/5], Step [600/600], Loss: 0.0478
Epoch [2/5], Step [100/600], Loss: 0.0722
Epoch [2/5], Step [200/600], Loss: 0.0601
Epoch [2/5], Step [300/600], Loss: 0.0159
Epoch [2/5], Step [400/600], Loss: 0.0793
Epoch [2/5], Step [500/600], Loss: 0.0472
Epoch [2/5], Step [600/600], Loss: 0.0435
Epoch [3/5], Step [100/600], Loss: 0.0092
Epoch [3/5], Step [200/600], Loss: 0.0113
Epoch [3/5], Step [300/600], Loss: 0.0760
Epoch [3/5], Step [400/600], Loss: 0.0210
Epoch [3/5], Step [500/600], Loss: 0.0267
Epoch [3/5], Step [600/600], Loss: 0.0408
Epoch [4/5], Step [100/600], Loss: 0.0254
Epoch [4/5], Step [200/600], Loss: 0.0805
Epoch [4/5], Step [300/600], Loss: 0.0064
Epoch [4/5], Step [400/600], Loss: 0.0384
Epoch [4/5], Step [500/600], Loss: 0.0021
Epoch [4/5], Step [600/600], Loss:

## Kiểm tra mô hình

In [10]:
# Ở giai đoạn kiểm tra mô hình, không cần tính đạo hàm nên sử dụng with torch.no_grad()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 98.9 %


## Lưu mô hình (model checkpoint)

In [11]:
torch.save(model.state_dict(), 'model.ckpt')